## 0. Environment

### 0.1 Install python libraries 
> Install pandas and openpyxl in a conda venv

SHELL
```shell
conda create --name pns_venv --yes
conda activate /workspaces/depressao-ibge-pns/depressao-ibge-pns/pns_venv
conda install pandas --yes
conda install openpyxl --yes
```

### 0.2 Decompress the pns-database and the dictionary
> The data is include in data/compressed-data.zip ~40MB<br>
> Decompresses it and will have two files summing up to ~500MB

SHELL
```shell
unzip data/compressed-data.zip -d data/ -x "__MACOSX/*"
```

In [ ]:
# Libraries
import pandas as pd
import numpy as np

## 1. Setup
- File paths
- Dictionary
- Column `_widths` and `_names`

In [21]:
# Paths
pns_data_path = "../data/pns-2019.txt"
dictionary_path = "../data/dicionario-limpo.xlsx"

In [22]:
dictionary = pd.read_excel(dictionary_path, skiprows=1)
dictionary.head()

,Posição inicial,Tamanho,Código\nda\nvariável,Quesito,Unnamed: 4,Categorias,Unnamed: 6
0,NaN,NaN,NaN,nº,descrição,Tipo,Descrição
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.0,2.0,V0001,NaN,Unidade da Federação,11,Rondônia
3,NaN,NaN,NaN,NaN,NaN,12,Acre
4,NaN,NaN,NaN,NaN,NaN,13,Amazonas


In [ ]:
# Widths
column_widths = dictionary.Tamanho.dropna(axis=0).values.astype(int)
column_widths

array([ 2,  7,  9,  4,  2,  4,  2,  1,  1,  1,  1,  1,  1,  1,  1,  2,  2,
        1,  1,  1,  1,  2,  2,  1,  1,  1,  1,  2,  1,  2,  1,  2,  1,  2,
        1,  2,  1,  2,  1,  2,  1,  2,  1,  2,  1,  1,  2,  1,  2,  2,  2,
        2,  2,  2,  1,  1,  1,  1,  2,  2,  2,  1,  2,  2,  4,  3,  1,  1,
        2,  1,  1,  1,  1,  1,  1,  3,  1,  1,  1,  1,  1,  2,  1,  2,  1,
        1,  1,  2,  1,  1,  1,  2,  1,  1,  1,  2,  1,  1,  1,  1,  1,  1,
        1,  1,  2,  2,  2,  1,  4,  1,  1,  1,  1,  5,  1,  8,  1,  8,  1,
        3,  1,  8,  1,  8,  1,  3,  1,  2,  2,  1,  2,  1,  2,  2,  2,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        3,  1,  1,  8,  1,  8,  1,  8,  1,  8,  1,  1,  1,  2,  1,  2,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  2,  2,  1,  1,  2,  1,
        1,  1,  2,  1,  3,  1,  1,  2,  1,  1,  1,  2,  2,  1,  1,  1,  1,
        1,  1,  2,  1,  1,  1,  1,  1,  1,  1,  2,  1,  2,  1,  2,  2,  1,
        1,  1,  1,  1,  1

In [ ]:
# Names
column_names = dictionary["Código\nda\nvariável"].dropna(axis=0)
column_names

2           V0001
29          V0024
30        UPA_PNS
31      V0006_PNS
32          V0015
          ...    
3835       VDF003
3837       VDF004
3845       VDM001
3850       VDP001
3854       VDDATA
Name: Código\nda\nvariável, Length: 816, dtype: object

## 2. Data Frame

### 2.1 FWF

In [ ]:
# FWF
df = pd.read_fwf(
    pns_data_path,
    header=None,
    dtype=str,
    widths=column_widths
)

In [28]:
df.shape

(293726, 816)

In [18]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,806,807,808,809,810,811,812,813,814,815
0,11,1110011,110000016,0001,01,2019,06,1,1,1,...,12,1,2,3,10912213,2061,NaN,NaN,NaN,NaN
1,11,1110011,110000016,0001,01,2019,06,1,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,11,1110011,110000016,0001,01,2019,06,1,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,11,1110011,110000016,0001,01,2019,06,1,1,9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,11,1110011,110000016,0001,01,2019,06,1,1,9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 2.2 Name DF columns with descriptions from the dictionary

In [81]:
# Descriptions
descriptions = dictionary.iloc[1:, 4].dropna(axis=0).str.strip()
descriptions.head()

2                    Unidade da Federação
29                                Estrato
30                                    UPA
31    Número de ordem do domicílio na PNS
32                     Tipo da entrevista
Name: Unnamed: 4, dtype: object

In [82]:
df.columns = descriptions
df.head()

Unnamed: 4,Unidade da Federação,Estrato,UPA,Número de ordem do domicílio na PNS,Tipo da entrevista,Ano de referência,Total de moradores,Tipo de situação censitária,Tipo de área,Seleção do morador de 15 anos ou mais para responder o questionário individual,...,Total de moradores com 15 anos ou mais,Nível de instrução mais elevado alcançado (pessoas de 5 anos ou mais de idade) padronizado para o Ensino Fundamental - SISTEMA DE 9 ANOS,Condição em relação à força de trabalho na semana de referência para pessoas de 14 anos ou mais de idade,Condição de ocupação na semana de referência para pessoas de 14 anos ou mais de idade,"Rendimento domiciliar \n(exclusive o rendimento das pessoas cuja condição na unidade domiciliar era pensionista, empregado doméstico ou parente do empregado doméstico)","Rendimento domiciliar per capita\n(exclusive o rendimento das pessoas cuja condição na unidade domiciliar era pensionista, empregado doméstico ou parente do empregado doméstico)","Faixa de rendimento domiciliar per capita (exclusive o rendimento das pessoas cuja condição na unidade domiciliar era pensionista, empregado doméstico ou parente do empregado doméstico)",Faixa de tempo gasto por dia no deslocamento casa-trabalho pelas pessoas ocupadas que se deslocavam para o trabalho,Tipo de cigarro industrializado comprado,"Data de geração do arquivo de microdados. Data ordenada na forma: ano (4 algarismos), mês (2) e dia (2) - AAAAMMDD"
0,11,1110011,110000016,0001,01,2019,06,1,1,1,...,12,1,2,3,10912213,2061,NaN,NaN,NaN,NaN
1,11,1110011,110000016,0001,01,2019,06,1,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,11,1110011,110000016,0001,01,2019,06,1,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,11,1110011,110000016,0001,01,2019,06,1,1,9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,11,1110011,110000016,0001,01,2019,06,1,1,9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 3. Attribute selection

### 3.1 Read attribute selection

In [167]:
# read data/attribute-selection.txt into a python str list
with open("../data/attribute-selection.txt", "r") as file:
    selected_attributes = [line.replace("\\n", " ").strip() for line in file.readlines()]

# show me the duplicates
duplicates = [item for item, count in zip(selected_attributes, np.bincount([selected_attributes.index(i) for i in selected_attributes])) if count > 1]
display(pd.DataFrame(duplicates))

# Remove duplicates from selected_attributes
selected_attributes = list(dict.fromkeys(selected_attributes))
pd.DataFrame(selected_attributes)

,0
0,Com quantos familiares ou parentes ___ pode co...
1,Com quantos amigos próximos ___ pode contar em...
2,Peso - Final (em kg) (3 inteiros e 1 casa deci...
3,Cônjuge ou companheiro(a) mora em nesse domicí...
4,Idade do morador na data de referência


,0
0,"Nas duas últimas semanas, com que frequência o..."
1,Algum médico ou profissional de saúde mental (...
2,Algum médico já lhe receitou algum medicamento...
3,Nas duas últimas semanas o(a) senhor(a) usou a...
4,Que idade o(a) Sr(a) tinha no primeiro diagnós...
5,O(A) Sr(a) vai ao médico/serviço de saúde regu...
6,Qual o principal motivo do(a) Sr(a) não visita...
7,Por causa da depressão Faz psicoterapia
8,Por causa da depressão Toma medicamentos
9,Algum dos medicamentos para depressão foi obti...


### 3.2 remove escape \n instances

In [170]:
# remove escaped sequences from the df.columns
df.columns = df.columns.str.replace(r"\n", "", regex=True).str.strip()

### 3.3 Depression V1

In [172]:
depression = df[selected_attributes]
display(depression.shape)
display(depression.head())

(293726, 56)

Unnamed: 4,"Nas duas últimas semanas, com que frequência o(a) Sr(a) se sentiu deprimido(a), “pra baixo” ou sem perspectiva?",Algum médico ou profissional de saúde mental (como psiquiatra ou psicólogo) já lhe deu o diagnóstico de depressão?,Algum médico já lhe receitou algum medicamento para depressão?,Nas duas últimas semanas o(a) senhor(a) usou algum medicamento para depressão?,Que idade o(a) Sr(a) tinha no primeiro diagnóstico de depressão?,O(A) Sr(a) vai ao médico/serviço de saúde regularmente por causa da depressão ou só quando tem algum problema?,Qual o principal motivo do(a) Sr(a) não visitar o médico/serviço de saúde regularmente por causa da depressão?,Por causa da depressão Faz psicoterapia,Por causa da depressão Toma medicamentos,Algum dos medicamentos para depressão foi obtido em serviço público de saúde?,...,Ano de nascimento,Idade do morador na data de referência,Unidade da Federação,Tipo de área,Tipo de situação censitária,Cor ou raça,Qual foi o curso mais elevado que ___frequentou,"Quem fez isso com você? (Se mais de uma pessoa, defina o pirncipal agressor)",Quem fez isso com você,Quem fez isso com você
0,NaN,1,1,2,2,NaN,2,1,1,NaN,...,1963,055,11,1,1,1,05,NaN,NaN,21
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1950,069,11,1,1,4,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1987,031,11,1,1,2,10,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,9999,009,11,1,1,2,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,9999,006,11,1,1,4,NaN,NaN,NaN,NaN


## 4. Remove null rows

In [176]:
# remove all rows with NaN in every the selected attributes
depression_cleaned = depression.dropna(how='all')
print(
    "Original", df.shape, "\n"
    "Depression", depression_cleaned.shape
)

Original (293726, 816) 
Depression (293726, 56)
